In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import ky_utils.ky_interpretable_functions as interpret
import ky_utils.ky_riskslim as slim
import ky_utils.ky_stumps as stumps

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from sklearn.metrics import roc_auc_score

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_six_month'].values

In [3]:
#### CART
depth = [5,6,7,8,9,10]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [60]
depth = [2]
learning_rate = [0.1]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                            learning_rate, 
                            depth, 
                            estimators, 
                            seed=816)

In [4]:
print("CART: ", np.mean(cart_summary['FL_score']))
print("EMB: ", np.mean(ebm_summary['FL_score']))

CART:  0.5626572387963321
EMB:  0.5710577295621853


---

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY-stumps.csv").sort_values('person_id')
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL-stumps.csv").sort_values('person_id')

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns[1:]

In [6]:
single_stump_model = stumps.stump_model(KY_X, KY_Y, FL_X, FL_Y, 0.0008, cols, 816)
## unique original features
unique_stumps = []
for i in single_stump_model['features']:
    unique_stumps.append(''.join([j for j in i if not j.isdigit()]))
print(len(np.unique(unique_stumps)))

15


In [7]:
c_grid={'C': [0.0005, 0.0008]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [13]:
print("CART: ", np.mean(cart_summary['FL_score']))
print("EMB: ", np.mean(ebm_summary['FL_score']))
print("Additive: ", np.mean(stumps_summary['FL_score']))

CART:  0.5626572387963321
EMB:  0.5710577295621853
Additive:  0.5738199018468294


---

### RiskSLIM

In [10]:
## train on best param chosen by Lasso Stumps from above
single_stump_model = stumps.stump_model(KY_X, 
                                        KY_Y, 
                                        FL_X, 
                                        FL_Y, 
                                        0.0005, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [11]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [12]:
riskslim_summary = slim.risk_cv(sub_KY_X, 
                                KY_Y, 
                                sub_FL_X, 
                                FL_Y, 
                                y_label = 'felony_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
04/06/20 @ 12:07 PM | 203 rows in lookup table
04/06/20 @ 12:07 PM | ------------------------------------------------------------
04/06/20 @ 12:07 PM | runnning initialization procedure
04/06/20 @ 12:07 PM | ------------------------------------------------------------
04/06/20 @ 12:07 PM | CPA produced 2 cuts
04/06/20 @ 12:07 PM | running naive rounding on 6 solutions
04/06/20 @ 12:07 PM | best objective value: 0.4414
04/06/20 @ 12:07 PM | rounding produced 5 integer solutions
04/06/20 @ 12:07 PM | best objective value is 0.3249
04/06/20 @ 12:07 PM | running sequential rounding on 6 solutions
04/06/20 @ 12:07 PM | best objective value: 0.4414
04/06/20 @ 12:07 PM | sequential rounding produced 5 integer solutions
04/06/20 @ 12:07 PM | best objective value: 0.2698
04/06/20 @ 12:07 PM | polishing 10 solutions
04/06/20 @ 12:07 PM | best objective value: 0.2698
04/06/20 @ 12:07 PM | polishing produced 5 integer solutions
04/06/20 @ 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/06/20 @ 12:08 PM | adding 265 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1022.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 157131 25961        cutoff              0.1000        0.0986   779508    1.42%           rho_0 N 157131 112613     39
 160736 25560        cutoff              0.1000        0.0986   793723    1.38%          rho_13 U 160736 160734     25
 164368 25409        cutoff              0.1000        0.0986   807804    1.35%           rho_1 N 164368 164367     33
Elapsed time = 45.49 sec. (41255.35 ticks, tree = 6.92 MB, solutions = 10)
 168089 25254        cutoff              0.1000        0.0987   822101    1.33%           rho_0 N 168089 168087     31
 171970 25679        cutoff              0.1000        0.0987   835192    1.30%          rho_13 U 171970  53230     33
 175733 25440        0.0995     7        0.1000        0.0987   849126    1.28%           rho_5 D 175733 175732     38
 179568 25735        cutoff              0.1000        0.0987   862907    1.25%           rho_0 D 179568 179567     30
 183332 25776        0.0993     9        0.1000        0.0988   877145    1.23%           rh

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/06/20 @ 12:09 PM | adding 268 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1015.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 171953 30154        0.0995     6        0.1001        0.0986   874979    1.48%           rho_9 D 171953 171951     32
Elapsed time = 42.95 sec. (41261.23 ticks, tree = 7.90 MB, solutions = 7)
 175995 30148        0.0990     5        0.1001        0.0987   888992    1.46%          rho_15 D 175995 175993     29
 179845 29932        0.0987    15        0.1001        0.0987   903442    1.43%           rho_5 U 179845 179843     17
 183510 29614        0.0987    13        0.1001        0.0987   917705    1.40%           rho_5 D 183510 183509     18
 187419 29704        0.0988    12        0.1001        0.0988   931649    1.37%           rho_1 U 187419 187412     26
 191541 29800        0.1000    16        0.1001        0.0988   944831    1.35%           rho_6 N 191541 111708     20
 195398 29421        0.1001     8        0.1001        0.0988   958440    1.30%           rho_9 U 195398 195397     21
 199127 29113        cutoff              0.1001        0.0989   972251    1.26%           rho

...

 188647 23294        0.0997    13        0.0998        0.0988   888596    0.96%           rho_0 U 188647 188646     21
 192850 22658        0.0992    11        0.0998        0.0988   903445    0.93%           rho_1 D 192850 192849     22
Elapsed time = 34.27 sec. (41257.71 ticks, tree = 6.20 MB, solutions = 9)
 197335 22671        0.0997    11        0.0998        0.0989   918292    0.89%           rho_2 N 197335  21476     30
 201537 21963        0.0989     8        0.0998        0.0989   932396    0.85%           rho_0 D 201537 201532     42
 205922 21350        cutoff              0.0998        0.0989   946147    0.81%          rho_14 U 205922 205921     33
 210294 20588        0.0991    11        0.0998        0.0990   959825    0.77%          rho_13 U 210294  55457     35
 214808 19493        0.0993     9        0.0998        0.0990   974415    0.71%           rho_0 U 214808 214807     31
 219521 18155        cutoff              0.0998        0.0991   988789    0.66%           rho

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/06/20 @ 12:33 PM | adding 256 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1016.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 225799 31722        cutoff              0.0992        0.0981   911151    1.14%        alpha_12 U 225799 106245     43
Elapsed time = 37.63 sec. (41256.00 ticks, tree = 9.48 MB, solutions = 7)
 230804 31585        0.0985     3        0.0992        0.0981   925626    1.10%        alpha_13 D 230804 216016     37
 235754 30875        0.0991     2        0.0992        0.0981   940409    1.07%         alpha_4 U 235754 235242     39
 241109 30864        0.0983     9        0.0992        0.0982   954198    1.05%           rho_5 U 241109 241108     38
 245896 29647        0.0984     7        0.0992        0.0982   967885    1.00%           rho_0 D 245896 245895     42
 251044 29694        0.0983    10        0.0992        0.0982   981530    0.97%           rho_1 U 251044 230435     24
 255660 28534        0.0985     8        0.0992        0.0983   996004    0.93%           rho_6 U 255660 112479     32
 260658 27456        cutoff              0.0992        0.0983  1010572    0.90%        alpha_

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
04/06/20 @ 12:34 PM | adding 254 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1021.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 191331  5348        cutoff              0.0997        0.0993   707227    0.35%           rho_7 U 191331  68194     40
 196497  4141        cutoff              0.0997        0.0994   718326    0.26%           rho_1 N 196497 196496     30
Elapsed time = 37.36 sec. (41256.41 ticks, tree = 2.22 MB, solutions = 8)
 201423  2490        0.0995     7        0.0997        0.0995   728632    0.17%           rho_0 D 201423 201420     33
 206800   785        cutoff              0.0997        0.0996   738943    0.06%           rho_5 D 206800 206798     30

User cuts applied:  374

Root node processing (before b&c):
  Real time             =    0.16 sec. (1.53 ticks)
Sequential b&c:
  Real time             =   38.67 sec. (43572.54 ticks)
                          ------------
Total (root+branch&cut) =   38.83 sec. (43574.07 ticks)
+----------------------------------------------+------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-6 + score))   |                  |           |
| ========

In [14]:
print("CART: ", np.mean(cart_summary['FL_score']))
print("EMB: ", np.mean(ebm_summary['FL_score']))
print("Additive: ", np.mean(stumps_summary['FL_score']))
print("Additive: ", np.mean(riskslim_summary['FL_score']))

CART:  0.5626572387963321
EMB:  0.5710577295621853
Additive:  0.5738199018468294
Additive:  0.549509879891515


In [15]:
#### save results
summary_felony6_ky_inter_model = {"cart": cart_summary,
                                  "ebm": ebm_summary,
                                  "stumps": stumps_summary,
                                  "riskslim": riskslim_summary}

In [16]:
path = "./results/interpretable/six-month/"
results = [["Felony",             
            np.str(round(np.mean(cart_summary['FL_score']),3)) + " (" + np.str(round(np.std(cart_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(ebm_summary['FL_score']), 3)) + " (" + np.str(round(np.std(ebm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(stumps_summary['FL_score']),3)) + " (" + np.str(round(np.std(stumps_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(riskslim_summary['FL_score']), 3)) + " (" + np.str(round(np.std(riskslim_summary['FL_score']),3)) + ")"]]
with open(path + 'six-month-ky-interpretable.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)